In [33]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

## Fine Food Reviews

Fuente: https://inclass.kaggle.com/c/fine-food-reviews

### Representacion

- **Id** - El id que identifica a cada review
- **ProductId** - El Id del producto
- **UserId** - El Id del usuario
- **ProfileName** - El nombre del usuario 
- **HelpfulnessNumerator** - El numerador indicando la cantidad de usuarios que juzgaron al review como util
- **HelpfulnessDenominator** - El denominador indicando la cantidad de usuarios que evaluaron si el review fue útil o no
- **Prediction** - La cantidad de estrellas del review
- **Time** - Un timestamp para el review
- **Summary** - Un resumen del review
- **Text** - Texto del review

In [34]:
## creamos el RDD a partir de
reviewsTextRDD = sc.textFile('data/fine-food-reviews-train.csv',8)
## aplicamos una accion para tomar los 15 items del RDD
reviewsTextRDD.take(1)

['"Id","ProductId","UserId","ProfileName","HelpfulnessNumerator","HelpfulnessDenominator","Prediction","Time","Summary","Text"']

### Parseando el CSV

In [40]:
def parse_csv(line):
    return re.sub("\"[^\"]*\"", lambda x: x.group(0).replace(',',''), line).split(',')

#can we do better?
reviewsRDD = reviewsTextRDD.map(parse_csv).filter(lambda a: a[0] != '"Id"')
reviewsRDD.take(1)

[['492346',
  '"B001VNP0Y6"',
  '"AEZRVY3CV52UZ"',
  '"Corey Wright ""Democratus"""',
  '3',
  '3',
  '5',
  '1320537600',
  '"Just what I needed"',
  '"I got them in a very timely manner and they\'re all very large and green. There seems to be an inordinate amount of the seeds that come out a weird beige color but it still tastes great and is a much better deal than any of the Indian markets in town. Cheers to Frontier."']]

### User Profiles con Mayor Cantidad de Reviews

In [43]:
from operator import add
reviewsRDD.map(lambda a: (a[3], 1)).reduceByKey(add).sortBy(ascending=False,keyfunc=lambda x:x[1]).take(10)

[('"C. F. Hill ""CFH"""', 347),
 ('"O. Brown ""Ms. O. Khannah-Brown"""', 338),
 ('"Gary Peterson"', 306),
 ('"Chris"', 292),
 ('"Rebecca of Amazon ""The Rebecca Review"""', 289),
 ('"Linda"', 224),
 ('"Mike"', 209),
 ('"c2"', 207),
 ('"Laura"', 206),
 ('"John"', 204)]

### ProductIds con Mayor Cantidad de Reviews

In [44]:
from operator import add
reviewsRDD.map(lambda a: (a[1], 1)).reduceByKey(add).sortBy(ascending=False,keyfunc=lambda x:x[1]).take(5)

[('"B007JFMH8M"', 714),
 ('"B003B3OOPA"', 511),
 ('"B0026RQTGE"', 509),
 ('"B002QWP8H0"', 508),
 ('"B002QWP89S"', 499)]

### Cantidad totales de Reviews por Calificacion

In [45]:
reviewsPerScoreRDD = reviewsRDD.map(lambda a: (a[6], 1)).reduceByKey(add).sortBy(ascending=False,keyfunc=lambda x:x[1])
reviewsPerScoreRDD.take(100)

[('5', 290403), ('4', 64593), ('1', 41775), ('3', 34118), ('2', 23871)]

### Probabilidad total de cada Calificacion

In [46]:
totalFreq = reviewsRDD.count()
reviewsPerScoreRDD.map(lambda x: (x[0], round(float(x[1])/totalFreq,3))).take(100)

[('5', 0.639), ('4', 0.142), ('1', 0.092), ('3', 0.075), ('2', 0.052)]

## Matriz de Utilidad

N usuarios con una lista de M items puede representarse como una matriz de NxM.
Similar a una matriz dispersa, pero con numero desconocidos (posibles clasificaciones).

Objetivo de un potencial clasificador es estimar las calificaciones que nos faltan, con un sistema de recomendacion.

